In [1]:
import tensorflow as tf

/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/y2536zha/anaconda3/envs/w

In [2]:
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from IPython.core.interactiveshell import InteractiveShell
from time import time
from copy import deepcopy
import pandas as pd

import matplotlib as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_rows', 100)

%matplotlib inline 
import sys, os
base_path = os.getcwd()[0:os.getcwd().rfind('Watermark')] + "Watermark/"
sys.path.append(base_path) 

import matplotlib.pyplot as plt
from src.asiaccs_main import asiaccs_blackbox
from src.models import get_deep_cnn_for_cifar, get_lenet_model_for_mnist
from src.preprocess_data import load_cifar_images, load_mnist_images, transform_to_one_hot
from src.util import plot_blackbox

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
import numpy as np

In [4]:
train, test = load_mnist_images()

In [5]:
x_train, y_train = train
x_test, y_test = test

x_train_original = x_train[:150000]
x_train_surrogate = x_train[150000:]

y_train_original = y_train[:150000]
y_train_surrogate = y_train[150000:]

del x_train
del y_train

x_train_original.shape
y_train_original.shape

x_train_surrogate.shape
y_train_surrogate.shape

x_test.shape
y_test.shape

(150000, 28, 28, 1)

(150000, 10)

(150000, 28, 28, 1)

(150000, 10)

(10000, 28, 28, 1)

(10000, 10)

In [6]:
model_original = get_lenet_model_for_mnist() 

In [7]:
model_original.load_weights('model/best_mnist_model.h5') # original model is trained on x_train_original only

In [8]:
model_original.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 500)               1225500   
_________________________________________________________________
fc2 (Dense)                  (None, 84)               

In [9]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import multiply, Input

In [10]:
def compute_jacob(x_train, y_train, model):
    y_label_byhost = Input((10,))
    output_masked = multiply([model.output, y_label_byhost])
    
    jcob = tf.gradients(ys = [output_masked],
                        xs = [model.input],)[0]
    
    tmp_fun = K.function([model.input, y_label_byhost],
                     [jcob])
    
    return tmp_fun([x_train, y_train])[0]
    

In [11]:
class TerminateOnBaseline(keras.callbacks.Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [12]:
x_train_attack = x_train_surrogate[:150]

In [13]:
y_train_attack = model_original.predict(x_train_attack).argmax(axis=1)
y_train_attack = transform_to_one_hot(y_train_attack, depth=10)

In [14]:
y_train_attack.shape
y_train_attack.shape
x_test.shape
y_test.shape

(150, 10)

(150, 10)

(10000, 28, 28, 1)

(10000, 10)

In [19]:
import gc
gc.collect()

2066

In [20]:
def Practical_black_box_attach(model_surrogate_fun, # function used to build the surrogate model
                               x_train_attack,
                               y_train_attack,
                               x_test,
                               y_test,
                               lambda_update = 1, # lamda used, as seen in the original paper
                               num_iter = 10,
                               eval_metric = 'acc',
                               delta = 0.1,
                               path_store_surrogate_model = 'model/best_mnist_model_surrogate_Black.h5',
                               batch_size_train = 64,
                               batch_size_eval = 1024,
                               epochs = 10):
    
    # Check Basic setting
    if eval_metric not in model_original.metrics_names:
        raise Exception('Error: eval_metric not in the original model')
        
    eval_metric_index = model_original.metrics_names.index(eval_metric)
    model_original_performance = model_original.evaluate(x_test, y_test, batch_size = batch_size_eval, verbose = 0)[eval_metric_index]
    
    print('Now show basic setting')
    print('original model performance:', eval_metric, model_original_performance)
    print('use', eval_metric, 'as the evaluation metric,', 'delta = ', delta,
      ',stealing will stop as long as the validation', eval_metric, 'achieve', model_original_performance - delta)
    
    
    # Preparation for training
    print('Now prepare for training')
    callbacks = TerminateOnBaseline(monitor = 'val_' + eval_metric, baseline = model_original_performance - delta)
    checkpoint_surrogate = keras.callbacks.ModelCheckpoint( path_store_surrogate_model, verbose=1, monitor='val_acc',save_best_only=True, mode='auto') 
    model_surrogate = model_surrogate_fun()
    
    # train model 
    history_list = []
    for i in range(num_iter):
        print('iteration:', i, 'x_train_attack shape:', x_train_attack.shape, 'y_train_attack shape:', y_train_attack.shape)
        history = model_surrogate.fit(x_train_attack,
                                      y_train_attack,
                                      batch_size = batch_size_train, 
                                      epochs = epochs, 
                                      verbose = 1,
                                      validation_data = (x_test, y_test),
                                      callbacks = [checkpoint_surrogate, callbacks])
        history_list.append(history)
        
        model_surrogate_performance = model_surrogate.evaluate(x_test, y_test, batch_size = batch_size_eval, verbose = 0)[eval_metric_index]
        if model_surrogate_performance >= model_original_performance - delta:
            print('Reached the desired performance, break the loop')
            break;

        # update synthetic dataset
        jacob = np.sign(compute_jacob(x_train_attack, y_train_attack, model_surrogate)) 
        x_train_attack_added = x_train_attack + lambda_update * jacob
        y_train_attack_added = model_original.predict(x_train_attack_added).argmax(axis=1)
        y_train_attack_added = transform_to_one_hot(y_train_attack_added, depth=10)

        x_train_attack = np.concatenate([x_train_attack, x_train_attack_added])
        y_train_attack = np.concatenate([y_train_attack, y_train_attack_added])
        
    return (model_surrogate, history_list)

In [26]:
import gc
gc.collect()

20

In [27]:
model_surrogate, history_list = Practical_black_box_attach(model_surrogate_fun = get_lenet_model_for_mnist,
                               x_train_attack = x_train_attack,
                               y_train_attack = y_train_attack,
                               x_test = x_test,
                               y_test = y_test,
                               lambda_update = 1,
                               num_iter = 10,
                               eval_metric = 'acc',
                               delta = 0.01,
                               path_store_surrogate_model = 'model/best_mnist_model_surrogate_Black.h5',
                               batch_size_train = 64,
                               batch_size_eval = 1024,
                               epochs = 20)

Now show basic setting
original model performance: acc 0.9924
use acc as the evaluation metric, delta =  0.01 ,stealing will stop as long as the validation acc achieve 0.9823999905586243
Now prepare for training
iteration: 0 x_train_attack shape: (150, 28, 28, 1) y_train_attack shape: (150, 10)
Train on 150 samples, validate on 10000 samples
Epoch 1/20
128/150 [========================>.....] - ETA: 0s - loss: 2.2668 - acc: 0.1953
Epoch 00001: val_acc improved from -inf to 0.14520, saving model to model/best_mnist_model_surrogate_Black.h5
150/150 [==============================] - 6s 38ms/sample - loss: 2.2520 - acc: 0.2067 - val_loss: 2.1760 - val_acc: 0.1452
Epoch 2/20
128/150 [========================>.....] - ETA: 0s - loss: 1.9668 - acc: 0.3047
Epoch 00002: val_acc improved from 0.14520 to 0.32260, saving model to model/best_mnist_model_surrogate_Black.h5
150/150 [==============================] - 4s 30ms/sample - loss: 1.9119 - acc: 0.3533 - val_loss: 1.9312 - val_acc: 0.3226
Epo

256/300 [========================>.....] - ETA: 0s - loss: 0.2371 - acc: 0.9219
Epoch 00007: val_acc did not improve from 0.70630
300/300 [==============================] - 5s 15ms/sample - loss: 0.2399 - acc: 0.9233 - val_loss: 1.0442 - val_acc: 0.6997
Epoch 8/20
256/300 [========================>.....] - ETA: 0s - loss: 0.1876 - acc: 0.9258
Epoch 00008: val_acc did not improve from 0.70630
300/300 [==============================] - 4s 15ms/sample - loss: 0.1965 - acc: 0.9200 - val_loss: 1.0458 - val_acc: 0.6995
Epoch 9/20
256/300 [========================>.....] - ETA: 0s - loss: 0.1335 - acc: 0.9648
Epoch 00009: val_acc improved from 0.70630 to 0.72500, saving model to model/best_mnist_model_surrogate_Black.h5
300/300 [==============================] - 5s 15ms/sample - loss: 0.1463 - acc: 0.9633 - val_loss: 0.9287 - val_acc: 0.7250
Epoch 10/20
256/300 [========================>.....] - ETA: 0s - loss: 0.0880 - acc: 0.9805
Epoch 00010: val_acc improved from 0.72500 to 0.72740, saving

Epoch 16/20
576/600 [===========================>..] - ETA: 0s - loss: 0.0017 - acc: 1.0000
Epoch 00016: val_acc did not improve from 0.76150
600/600 [==============================] - 5s 8ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 0.9529 - val_acc: 0.7562
Epoch 17/20
576/600 [===========================>..] - ETA: 0s - loss: 0.0015 - acc: 1.0000
Epoch 00017: val_acc did not improve from 0.76150
600/600 [==============================] - 5s 8ms/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 0.9628 - val_acc: 0.7559
Epoch 18/20
576/600 [===========================>..] - ETA: 0s - loss: 0.0013 - acc: 1.0000
Epoch 00018: val_acc did not improve from 0.76150
600/600 [==============================] - 5s 9ms/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 0.9678 - val_acc: 0.7565
Epoch 19/20
576/600 [===========================>..] - ETA: 0s - loss: 0.0012 - acc: 1.0000
Epoch 00019: val_acc did not improve from 0.76150
600/600 [==============================] - 5s 8ms/sample - loss

2368/2400 [============================>.] - ETA: 0s - loss: 0.0623 - acc: 0.9789
Epoch 00004: val_acc did not improve from 0.79850
2400/2400 [==============================] - 7s 3ms/sample - loss: 0.0628 - acc: 0.9783 - val_loss: 0.7018 - val_acc: 0.7907
Epoch 5/20
2368/2400 [============================>.] - ETA: 0s - loss: 0.0416 - acc: 0.9873
Epoch 00005: val_acc did not improve from 0.79850
2400/2400 [==============================] - 7s 3ms/sample - loss: 0.0416 - acc: 0.9875 - val_loss: 0.8515 - val_acc: 0.7621
Epoch 6/20
2368/2400 [============================>.] - ETA: 0s - loss: 0.0247 - acc: 0.9928
Epoch 00006: val_acc improved from 0.79850 to 0.80680, saving model to model/best_mnist_model_surrogate_Black.h5
2400/2400 [==============================] - 7s 3ms/sample - loss: 0.0245 - acc: 0.9929 - val_loss: 0.6812 - val_acc: 0.8068
Epoch 7/20
2368/2400 [============================>.] - ETA: 0s - loss: 0.0165 - acc: 0.9949
Epoch 00007: val_acc improved from 0.80680 to 0.823

Epoch 14/20
4736/4800 [============================>.] - ETA: 0s - loss: 0.0166 - acc: 0.9962
Epoch 00014: val_acc did not improve from 0.82300
4800/4800 [==============================] - 10s 2ms/sample - loss: 0.0165 - acc: 0.9960 - val_loss: 1.0137 - val_acc: 0.7575
Epoch 15/20
4736/4800 [============================>.] - ETA: 0s - loss: 0.0151 - acc: 0.9962
Epoch 00015: val_acc did not improve from 0.82300
4800/4800 [==============================] - 9s 2ms/sample - loss: 0.0151 - acc: 0.9960 - val_loss: 1.1223 - val_acc: 0.7404
Epoch 16/20
4736/4800 [============================>.] - ETA: 0s - loss: 0.0127 - acc: 0.9960
Epoch 00016: val_acc did not improve from 0.82300
4800/4800 [==============================] - 9s 2ms/sample - loss: 0.0125 - acc: 0.9960 - val_loss: 1.1655 - val_acc: 0.7478
Epoch 17/20
4736/4800 [============================>.] - ETA: 0s - loss: 0.0025 - acc: 0.9996
Epoch 00017: val_acc did not improve from 0.82300
4800/4800 [==============================] - 10s

19136/19200 [============================>.] - ETA: 0s - loss: 0.0450 - acc: 0.9852
Epoch 00003: val_acc did not improve from 0.82300
19200/19200 [==============================] - 26s 1ms/sample - loss: 0.0450 - acc: 0.9852 - val_loss: 1.3314 - val_acc: 0.7392
Epoch 4/20
19136/19200 [============================>.] - ETA: 0s - loss: 0.0312 - acc: 0.9914
Epoch 00004: val_acc did not improve from 0.82300
19200/19200 [==============================] - 26s 1ms/sample - loss: 0.0312 - acc: 0.9914 - val_loss: 1.2323 - val_acc: 0.7354
Epoch 5/20
19136/19200 [============================>.] - ETA: 0s - loss: 0.0165 - acc: 0.9959
Epoch 00005: val_acc did not improve from 0.82300
19200/19200 [==============================] - 26s 1ms/sample - loss: 0.0165 - acc: 0.9959 - val_loss: 1.4192 - val_acc: 0.7606
Epoch 6/20
19136/19200 [============================>.] - ETA: 0s - loss: 0.0216 - acc: 0.9937
Epoch 00006: val_acc did not improve from 0.82300
19200/19200 [==============================] - 

In [25]:
import gc
gc.collect()

433

In [24]:
model_surrogate.evaluate(x_test, y_test, batch_size = 1024)

10000/10000 [==============================] - 1s 83us/sample - loss: 1.1226 - acc: 0.7063


[1.1225852348327636, 0.7063]